In [1]:
import os
import random
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, UnidentifiedImageError
from tempfile import TemporaryDirectory
import pandas as pd
import csv

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, random_split

import torchvision
from torchvision import datasets, models, transforms

In [30]:
# Load dataset from folder
class FolderDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        """
        Initialize the FolderDataset.

        Args:
            data_dir (str): Path to the dataset directory containing subdirectories for each class.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data_dir = data_dir
        self.transform = transform
        self.img_names, self.image_paths = self._load_data()

    def _load_data(self):
        """
        Load data by iterating through each class subdirectory and collecting image paths and labels.

        Returns:
            tuple: A tuple containing lists of image names, image paths, and corresponding labels.
        """
        image_paths = []
        img_names = []
        

        for img_path in os.listdir(self.data_dir):
            img_name = int(os.path.splitext(img_path)[0])
            img_path = os.path.join(self.data_dir, img_path)
            img_names.append(img_name)
            image_paths.append(img_path)
        return img_names, image_paths

    def __len__(self):
        """
        Get the length of the dataset.

        Returns:
            int: Number of samples in the dataset.
        """
        return len(self.image_paths)

    def __getitem__(self, idx):
        """
        Retrieve a sample from the dataset by index.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            tuple: A tuple containing the image name, image data, and corresponding label.
        """
        img_name = self.img_names[idx]
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
        except FileNotFoundError:
            print(f"File not found: {img_path}")
            sys.exit(1)  # Exit the program with an error code
        except UnidentifiedImageError:
            print(f"Cannot identify image file: {img_path}")
            sys.exit(1)  # Exit the program with an error code
        except Exception as e:
            print(f"Unexpected error loading image {img_path}: {e}")
            sys.exit(1)  # Exit the program with an error code

        if self.transform:
            try:
                image = self.transform(image)
            except Exception as e:
                print(f"Error applying transform to image {img_path}: {e}")
                raise
        return img_name, image


In [3]:
# Data transformation
transform = transforms.Compose([
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [4]:
# Location of data and csv file
data_dir = '/kaggle/input/wb-data/wb_recognition_dataset/val/images'

In [31]:
# Test dataset (use suitable Dataset class)
test_dataset = FolderDataset(data_dir, transform)

In [32]:
# Dataloader iterators
test_dataloader = DataLoader(test_dataset, batch_size = 32, shuffle = False, num_workers=4, pin_memory = True)

In [14]:
# Check number of images and labels
test_dataset_size = len(test_dataset)
print('Number of images in test dataset: ', len(test_dataset))


Number of images in test dataset:  1392


In [15]:
# Number of classes
num_classes = 2139  

In [16]:
pip install efficientnet-pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=a025e64b856f90de8d2dd2a97f44edf41fdfa2424a23d9d0840a235eb5a26bb5
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch
Note: you may need to restart the kernel to use updated packages.


In [18]:
# Load pretrained model
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0')
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, num_classes)

# Move to gpu 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [28]:
def test_model(model, dataloader, csv_result):
    # Set the model to evaluation mode
    model.eval()

    # A list to store the results
    results = []

    for image_names, inputs in dataloader:
        inputs = inputs.to(device)

        # Get the predicted labels
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # Store the image names and predicted labels in the results list
        for image_name, pred in zip(image_names, preds):
            # use image_name.item() for csv
            results.append([image_name.item(), pred.item()])
    
    results = sorted(results, key=lambda x: x[0]) 
            
    # Save the results to a CSV file 
    try:
        with open(csv_result, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['image_name', 'label'])
            writer.writerows(results)
        print(f'Results saved successfully to {csv_result}')
    except Exception as e:
        print(f'Error saving results to CSV: {e}')            

In [20]:
# Location of saved model 
saved_model_path = '/kaggle/input/efficientnetb0-1/pytorch/efficientnet_b0_1/4/efficientNetb0-imagenet-11-best.pt'

In [21]:
try:
    if torch.cuda.is_available():
        model.load_state_dict(torch.load(saved_model_path))
    else:
        model.load_state_dict(torch.load(saved_model_path, map_location=torch.device('cpu')))
    print('Loaded saved model successfully')
except FileNotFoundError:
    print('File not found')
except Exception as e:
    print(f'An error occurred: {e}')

Loaded saved model successfully


In [22]:
csv_result_path = '/kaggle/working/result_folder.csv'

In [33]:
try:
    test_model(model, test_dataloader, csv_result_path)
except Exception as e:
    print(f'An error occurred during model testing: {e}')
    

Results saved successfully to /kaggle/working/result_folder.csv
